In [3]:
import json
from datasets import SemEvalDataSet
from transformers import AutoTokenizer
from label_mappings import SENTENCE_B

In [5]:
file_path = 'data/semeval2016/ABSA16_Restaurants_Train_SB1_v2.xml'
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased', cache_dir='../models/bert-base-cased')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
dataset = SemEvalDataSet(file_path, tokenizer, sentence_b=SENTENCE_B['uncased'])

In [42]:
def merge_origin_sentences(dataset, num_merge):
    count = 0
    merge_result = []
    for idx, (text, triplets) in enumerate(dataset.string_sentences):
        merge_sentences = [(text, triplets)]
        if idx < len(dataset.string_sentences) - num_merge:
            for i in range(1, num_merge + 1):
                merge_sentences.append(dataset.string_sentences[idx + i])

        start = 0
        aspects = set()
        cur_merge_triplets = []
        cls_triplets = []
        aug_text = ""
        for idx, (merge_text, merge_triplets) in enumerate(merge_sentences):
            if idx == 0:
                aug_text += merge_text
                add = 0
            else:
                aug_text += " " + merge_text
                add = 1
            merge_triplets = json.loads(merge_triplets)
            for m_t in merge_triplets:
                new_t = m_t.copy()
                if new_t['target'] == "NULL":
                    cls_triplets.append(new_t)
                    continue
                new_t['from'] = int(new_t['from']) + add + start
                new_t['to'] = int(new_t['to']) + add + start
                assert aug_text[new_t['from']: new_t['to']] == new_t['target'], "{} {}".format(aug_text[new_t['from']: new_t['to']], new_t['target'])
                cur_merge_triplets.append(new_t)
                aspects.add(new_t['category'])
            start += len(merge_text) + add
        for cls_t in cls_triplets:
            if cls_t['category'] in aspects:
                continue
            cur_merge_triplets.append(cls_t)
            aspects.add(cls_t['category'])

        merge_result.append((aug_text, cur_merge_triplets))
    return merge_result

In [46]:
merge_result = merge_origin_sentences(dataset, 2)

In [47]:
merge_result

[('Judging from previous posts this used to be a good place, but not any longer. We, there were four of us, arrived at noon - the place was empty - and the staff acted like we were imposing on them and they were very rude. They never brought us complimentary noodles, ignored repeated requests for sugar, and threw our dishes on the table. The food was lousy - too sweet or too salty and the portions tiny.',
  [{'target': 'place',
    'category': 'RESTAURANT#GENERAL',
    'polarity': 'negative',
    'from': 51,
    'to': 56},
   {'target': 'staff',
    'category': 'SERVICE#GENERAL',
    'polarity': 'negative',
    'from': 153,
    'to': 158},
   {'target': 'food',
    'category': 'FOOD#QUALITY',
    'polarity': 'negative',
    'from': 341,
    'to': 345},
   {'target': 'portions',
    'category': 'FOOD#STYLE_OPTIONS',
    'polarity': 'negative',
    'from': 389,
    'to': 397}]),
 ('We, there were four of us, arrived at noon - the place was empty - and the staff acted like we were imposin

In [11]:
a = np.ones((32, 128))

In [15]:
np.repeat(a[:, np.newaxis, :], a.shape[1], axis=1).shape

(32, 128, 128)

In [14]:
a.shape

(32, 128)